In [ ]:
!git clone https://github.com/DANUSHKUMARS/danush.git

Cloning into 'danush'...
remote: Enumerating objects: 6407, done.
remote: Counting objects: 100% (6407/6407), done.
remote: Compressing objects: 100% (6407/6407), done.
remote: Total 6407 (delta 0), reused 6407 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6407/6407), 32.95 MiB | 15.65 MiB/s, done.


In [ ]:
!ls danush/Alzheimer_MRI_4_classes_dataset/MildDemented

'10 (10).jpg'  '14 (27).jpg'  '19 (5).jpg'   '24 (19).jpg'  '29 (24).jpg'  '4 (5).jpg'
'10 (11).jpg'  '14 (28).jpg'  '19 (6).jpg'   '24 (20).jpg'  '29 (25).jpg'  '4 (6).jpg'
'10 (12).jpg'  '14 (2).jpg'   '19 (7).jpg'   '24 (21).jpg'  '29 (26).jpg'  '4 (7).jpg'
'10 (13).jpg'  '14 (3).jpg'   '19 (8).jpg'   '24 (22).jpg'  '29 (27).jpg'  '4 (8).jpg'
'10 (14).jpg'  '14 (4).jpg'   '19 (9).jpg'   '24 (23).jpg'  '29 (28).jpg'  '4 (9).jpg'
'10 (15).jpg'  '14 (5).jpg'    19.jpg	     '24 (24).jpg'  '29 (2).jpg'    4.jpg
'10 (16).jpg'  '14 (6).jpg'   '1 (9).jpg'    '24 (25).jpg'  '29 (3).jpg'   '5 (10).jpg'
'10 (17).jpg'  '14 (7).jpg'    1.jpg	     '24 (26).jpg'  '29 (4).jpg'   '5 (11).jpg'
'10 (18).jpg'  '14 (8).jpg'   '20 (10).jpg'  '24 (27).jpg'  '29 (5).jpg'   '5 (12).jpg'
'10 (19).jpg'  '14 (9).jpg'   '20 (11).jpg'  '24 (28).jpg'  '29 (6).jpg'   '5 (13).jpg'
'10 (20).jpg'   14.jpg	      '20 (12).jpg'  '24 (2).jpg'   '29 (7).jpg'   '5 (14).jpg'
'10 (21).jpg'  '1 (4).jpg'    '20 (13).jpg'  '24 

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:
dataset_path = './danush/Alzheimer_MRI_4_classes_dataset/'

In [ ]:
categories = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
image_paths = []
labels = []

for category in categories:
    category_path = os.path.join(dataset_path, category)
    for img in os.listdir(category_path):
        image_paths.append(os.path.join(category_path, img))
        labels.append(category)

# Convert to numpy arrays
image_paths = np.array(image_paths)
labels = np.array(labels)


In [ ]:
# First split into training and testing sets (15% for testing)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    image_paths, labels, test_size=0.15, random_state=42, stratify=labels
)

# Then split the training+validation set into training and validation sets (15% for validation)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val
)  # 0.1765 is approximately 15% of the original dataset


Explanation of the Splitting Logic
First Split (Training + Validation and Testing):

The first call to train_test_split takes the full dataset and sets aside 15% for the testing set. This leaves 85% for training and validation.
Second Split (Training and Validation):

The second split takes the remaining 85% and divides it into 70% for training and 15% for validation.
The test_size=0.1765 parameter is calculated to represent roughly 15% of the total dataset size. When you take 15% of the 85%, it translates to about 12.75% of the original dataset, leaving you with 70% for training.

In [ ]:
IMG_SIZE = (224, 224)  # Image size
BATCH_SIZE = 200         # Batch size

# Create a data generator for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,               # Normalize pixel values
    validation_split=0.15,        # 15% of the training data will be used for validation
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Load training data
train_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_train, 'class': y_train}),
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Load validation data
val_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_val, 'class': y_val}),
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Load test data without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_test, 'class': y_test}),
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Do not shuffle test data
)


Found 4479 validated image filenames belonging to 4 classes.
Found 961 validated image filenames belonging to 4 classes.
Found 960 validated image filenames belonging to 4 classes.
